In [1]:
import csv
import tensorflow as tf
import numpy as np
import wget
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional


Importing the Dataset

In [2]:

!wget --no-check-certificate \
    https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv \
    -O C:\Users\a2sha\Documents\MachineLearning\MachineLearningAssignment4\bbc-text.csv
    #-O /tmp/bbc-text.csv

--2022-03-31 17:39:30--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.72.144, 142.250.72.176, 142.251.40.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.72.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: 'C:/Users/a2sha/Documents/MachineLearning/MachineLearningAssignment4/bbc-text.csv'

     0K .......... .......... .......... .......... ..........  1% 2.30M 2s
    50K .......... .......... .......... .......... ..........  2% 4.65M 2s
   100K .......... .......... .......... .......... ..........  3% 4.46M 1s
   150K .......... .......... .......... .......... ..........  4% 2.54M 1s
   200K .......... .......... .......... .......... ..........  5% 4.49M 1s
   250K .......... .......... .......... .......... ..........  6% 4.04M 1s
   300K .......... .......... .......... .......... ..........  7% 5.3

Setting up nltk and stopwords

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))


  1850K .......... .......... .......... .......... .......... 38% 11.7M 1s
  1900K .......... .......... .......... .......... .......... 39% 8.74M 1s
  1950K .......... .......... .......... .......... .......... 40% 5.92M 1s
  2000K .......... .......... .......... .......... .......... 41% 8.08M 1s
  2050K .......... .......... .......... .......... .......... 42% 9.61M 1s
  2100K .......... .......... .......... .......... .......... 43% 7.08M 1s
  2150K .......... .......... .......... .......... .......... 44% 10.3M 1s
  2200K .......... .......... .......... .......... .......... 45% 4.84M 1s
  2250K .......... .......... .......... .......... .......... 46% 5.82M 1s
  2300K .......... .......... .......... .......... .......... 47% 4.58M 0s
  2350K .......... .......... .......... .......... .......... 48% 3.39M 0s
  2400K .......... .......... .......... .......... .......... 49% 3.91M 0s
  2450K .......... .......... .......... .......... .......... 50% 2.72M 0s
  2500K ...

Set Hyper-Parameters

In [4]:
vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary
training_portion = .8

Populate the list of articles and labels from the data and also remove the stopwords.

In [5]:
articles = []
labels = []

with open("C:/Users/a2sha/Documents/MachineLearning/MachineLearningAssignment4/bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        article = row[1]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)

Creating train and validation set

In [6]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

Tokenize and Sequenize

In [7]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_articles)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


Setting up labels

In [8]:

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

Creating the model

In [9]:
model=Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 6)                 774       
                                                                 
Total params: 386,822
Trainable params: 386,822
Non-trainable params: 0
_________________________________________________________________


Compile

In [10]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

In [11]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, 
                    validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/10
56/56 - 26s - loss: 1.5919 - accuracy: 0.2899 - val_loss: 1.5724 - val_accuracy: 0.2697 - 26s/epoch - 457ms/step
Epoch 2/10
56/56 - 12s - loss: 1.1817 - accuracy: 0.5511 - val_loss: 0.7590 - val_accuracy: 0.8404 - 12s/epoch - 221ms/step
Epoch 3/10
56/56 - 13s - loss: 0.6072 - accuracy: 0.8236 - val_loss: 0.6074 - val_accuracy: 0.7978 - 13s/epoch - 225ms/step
Epoch 4/10
56/56 - 12s - loss: 0.3499 - accuracy: 0.9292 - val_loss: 0.4261 - val_accuracy: 0.8854 - 12s/epoch - 217ms/step
Epoch 5/10
56/56 - 13s - loss: 0.1963 - accuracy: 0.9483 - val_loss: 0.3662 - val_accuracy: 0.9011 - 13s/epoch - 225ms/step
Epoch 6/10
56/56 - 13s - loss: 0.1453 - accuracy: 0.9764 - val_loss: 0.2600 - val_accuracy: 0.9281 - 13s/epoch - 226ms/step
Epoch 7/10
56/56 - 12s - loss: 0.0547 - accuracy: 0.9955 - val_loss: 0.1976 - val_accuracy: 0.9461 - 12s/epoch - 211ms/step
Epoch 8/10
56/56 - 11s - loss: 0.0208 - accuracy: 0.9978 - val_loss: 0.2015 - val_accuracy: 0.9416 - 11s/epoch - 197ms/step
Epoch 9/

Prediction Examples

In [12]:

txt = ["blair prepares to name poll date tony blair is likely to name 5 may as election day when parliament returns from its easter break  the bbc s political editor has learned.  andrew marr says mr blair will ask the queen on 4 or 5 april to dissolve parliament at the end of that week. mr blair has so far resisted calls for him to name the day but all parties have stepped up campaigning recently. downing street would not be drawn on the claim  saying election timing was a matter for the prime minister.  a number 10 spokeswoman would only say:  he will announce an election when he wants to announce an election.  the move will signal a frantic week at westminster as the government is likely to try to get key legislation through parliament. the government needs its finance bill  covering the budget plans  to be passed before the commons closes for business at the end of the session on 7 april.  but it will also seek to push through its serious and organised crime bill and id cards bill. mr marr said on wednesday s today programme:  there s almost nobody at a senior level inside the government or in parliament itself who doesn t expect the election to be called on 4 or 5 april.  as soon as the commons is back after the short easter recess  tony blair whips up to the palace  asks the queen to dissolve parliament ... and we re going.  the labour government officially has until june 2006 to hold general election  but in recent years governments have favoured four-year terms."]

seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = ['sport', 'bussiness', 'politics', 'tech', 'entertainment'] 

print(pred)
print(np.argmax(pred))
print(labels[np.argmax(pred)-1])

[[1.5735817e-04 2.6085167e-04 3.2820776e-03 9.9299192e-01 1.1172922e-03
  2.1904306e-03]]
3
politics


In [13]:

txt = ["call to save manufacturing jobs the trades union congress (tuc) is calling on the government to stem job losses in manufacturing firms by reviewing the help it gives companies.  the tuc said in its submission before the budget that action is needed because of 105 000 jobs lost from the sector over the last year. it calls for better pensions  child care provision and decent wages. the 36-page submission also urges the government to examine support other european countries provide to industry. tuc general secretary brendan barber called for  a commitment to policies that will make a real difference to the lives of working people.    greater investment in childcare strategies and the people delivering that childcare will increases the options available to working parents   he said.  a commitment to our public services and manufacturing sector ensures that we can continue to compete on a global level and deliver the frontline services that this country needs.  he also called for  practical measures  to help pensioners  especially women who he said  are most likely to retire in poverty . the submission also calls for decent wages and training for people working in the manufacturing sector."]

seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = ['sport', 'bussiness', 'politics', 'tech', 'entertainment'] 

print(pred)
print(np.argmax(pred))
print(labels[np.argmax(pred)-1])

[[1.1402832e-04 2.4513642e-03 9.7615951e-01 2.0509444e-02 9.6237731e-05
  6.6940807e-04]]
2
bussiness


I can see that it correctly predicted them as politics and business